In [2]:
import pandas as pd
import numpy as np
import re
from glob import glob

In [3]:
clinics = ["Ballard","BEAV","Burlingame","CherryCreek","DC","DEN","Kona","LO","MidCity","NOLA","NorthPortland","Pearl","PORT","SAR","SFEC","TANAS","VANC","ProgressRidge", "QueenAnne", "PowellSquare", "HappyValley", "Sunnyvale"] #Progrdige placeholder
# clinics = ['VANC'] #useful for testing modifications
code_mix_df = pd.DataFrame(columns=['97530','97535','97112','97001','97162','97163','97161','97002','97164','92542,64550,29280,29520,29530,29550,29200,20552','97799,20053','97750'])
df = pd.DataFrame(columns=["PT","All",'97530','97535','97112','97110','97140','97001','97162','97163','97161','97002','97164','98960','97116','97033','97760','97150','97124','97039','97035','97032','97016','97014','97012','97010','95853','95852','95851','95597,"G0283","G0282",20553','97608','92542','95992,29540','97610,64550,29240,29260,29280,29520,29530,29550,29200,29799,20552','97799,20053','97597','97750','97000','97774','97775','97776','97777','97546,"NC001"'])

In [4]:
for i in clinics:
    import glob
    glob = glob.glob(i+"*.xlsx")
    df2 = pd.concat(pd.read_excel(f) for f in glob)
    df2['CPT'] = df2['CPT'].str.replace('GP:','')
    df2['CPT'] = df2['CPT'].str.replace(':59','')
    df2['CPT'] = df2['CPT'].str.replace('.KX','')
    df2['PT'] = df2['PT'].str.replace(r'[A-Z][A-Z][A-Z] \(','')
    df2['PT'] = df2['PT'].str.replace(r'[A-Z][A-Z] \(','')
    df2['PT'] = df2['PT'].str.replace(')','')


    
    pivot = np.round(pd.pivot_table(df2, values= 'Units',
                                index=["PT","Patient", "Date of Service"],
                                columns=["CPT"],
                                aggfunc="sum",
                                margins=True    
                                ))

    df3 = pivot.reset_index().reindex(columns = df.columns).replace(np.nan, 0)
    df3 = df3[df3['PT'] != 'All']

    clinic_reval = df3[['97161','97162','97163','97164']].sum()
    clinic_reval = clinic_reval['97164']/(clinic_reval['97161']+clinic_reval['97162']+ clinic_reval['97163'])
    
    clinic_total_eval = df3.loc[(df3.get('97161') == 1.0) | (df3.get('97162') == 1.0) | (df3['97163'] == 1.0) & (df3["All"] <=10)].sum()
    clinic_total_eval["Eval/CPT"] = clinic_total_eval["All"]/(clinic_total_eval.get('97161')+clinic_total_eval.get('97162')+clinic_total_eval.get('97163'))
    clinic_total_eval = clinic_total_eval["Eval/CPT"]
    
    clinic_non = df3.loc[(df3.get('97161') == 0.0) & (df3.get('97162') == 0.0) & (df3.get('97163') == 0.0)].reset_index()
    clinic_non["Noneval/CPT"] = (clinic_non["All"].sum()/clinic_non.shape[0])
    clinic_non = clinic_non["Noneval/CPT"]
    clinic_non = clinic_non.drop_duplicates()
    
    clinic_code_mix_df = df3.reindex(columns=code_mix_df.columns)
    clinic_code_mix_x = clinic_code_mix_df.sum()
    clinic_code_mix_x = clinic_code_mix_x.sum()
    clinic_code_mix = clinic_code_mix_x/df3["All"].sum()


    with open(' CPT Analysis.txt', 'a') as file_object:
        file_object.write(
            f"=========================\n{i}\n=========================\n"
            +f"Clinic Code Mix: {clinic_code_mix:.0%}\n"
            +f"Clinic Codes per Eval: {clinic_total_eval:.3}\n"
            +f"Clinic Codes per Non-Eval: {pd.to_numeric(clinic_non.to_string(index=False)):.3}\n"
            +f"Clinic Re-eval per Eval: {clinic_reval:.0%}\n\n"
            # +f"Eval Total: {clinic_eval_total} \nVisit Total: {clinic_visit_total}\n\n"
        )

    pt = df3.PT.unique()


    for g in pt:
        df4 = df3.loc[(df3["PT"].str.contains(g,regex =False))]
        df4['PT'] = g 

        reval = df4[["PT",'97161','97162','97163','97164']].groupby("PT").sum().reset_index()
        reval['97164'] = reval['97164'].replace(np.nan, 1)
        reval['Reval'] = reval['97164']/(reval['97161'] + reval['97162']+ reval['97163'])
        reval['Reval'] = reval['Reval'].replace(np.nan, 0)

        eval = df4.loc[(df4.get('97161') == 1.0) | (df4.get('97162') == 1.0) | (df4.get('97163') == 1.0)].groupby("PT").sum().reset_index()
        eval['Eval/CPT'] = eval["All"]/(eval.get('97161')+eval.get('97162')+eval.get('97163'))

        non = df4.loc[(df4.get('97161') == 0.0) & (df4.get('97162') == 0.0) & (df4.get('97163') == 0.0)].reset_index()
        non["Noneval/CPT"] = non["All"].sum()/non.shape[0]
        non = non[["PT","Noneval/CPT"]]
        non = non.drop_duplicates()
    
        cm_df = df4.reindex(columns=code_mix_df.columns)
        cm_df_x = cm_df.sum()
        cm_df_x = cm_df_x.sum()
        cm = cm_df_x/df4["All"].sum()

        with open(' CPT Analysis.txt', 'a') as file_object:
            file_object.write(
            f"{g} Code Mix: {cm:.0%}\n"
            +f"{g} Codes per Eval: {(eval['Eval/CPT'].to_string(index=False)):.4}\n"
            +f"{g} Codes per Non-Eval: {(non['Noneval/CPT'].to_string(index=False)):.4}\n"
            +f"{g} Reval per Eval: {pd.to_numeric(reval['Reval'].to_string(index=False)):.0%}\n\n"
        )

C:\Users\Madias\AppData\Local\Temp\ipykernel_69324\2499258874.py:7: FutureWarning: The default value of regex will change from True to False in a future version.
  df2['CPT'] = df2['CPT'].str.replace('.KX','')
C:\Users\Madias\AppData\Local\Temp\ipykernel_69324\2499258874.py:8: FutureWarning: The default value of regex will change from True to False in a future version.
  df2['PT'] = df2['PT'].str.replace(r'[A-Z][A-Z][A-Z] \(','')
C:\Users\Madias\AppData\Local\Temp\ipykernel_69324\2499258874.py:9: FutureWarning: The default value of regex will change from True to False in a future version.
  df2['PT'] = df2['PT'].str.replace(r'[A-Z][A-Z] \(','')
C:\Users\Madias\AppData\Local\Temp\ipykernel_69324\2499258874.py:10: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df2['PT'] = df2['PT'].str.replace(')','')
C:\Users\Madias\AppData\Local\Te